In [26]:
import pandas as pd
import sqlite3
import networkx as nx
import numpy as np
import json

In [ ]:
conn = sqlite3.connect('GTEx.db')
df = pd.read_sql("SELECT * FROM GTEx_network_age_adjusted", conn)


G = nx.from_pandas_edgelist(df, 'row', 'col', ['weight'])

# Now use graph algorithms
conn.close()

24813


In [10]:
print([len(subgraph) for subgraph in nx.connected_components(G)])

[24809, 2, 2]


In [3]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('GTEx.db')

# How many edges total?
total = pd.read_sql("SELECT COUNT(*) as count FROM GTEx_network_age_adjusted", conn)
print(f"Total edges: {total['count'][0]:,}")

# How many unique genes?
genes = pd.read_sql("""
    SELECT COUNT(DISTINCT gene) as count FROM (
        SELECT row as gene FROM GTEx_network_age_adjusted
        UNION
        SELECT col as gene FROM GTEx_network_age_adjusted
    )
""", conn)
print(f"Unique genes: {genes['count'][0]:,}")

# Distribution by weight
for threshold in [0.25, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.97, 0.99]:
    count = pd.read_sql(f"""
        SELECT COUNT(*) as count 
        FROM GTEx_network_age_adjusted 
        WHERE weight > {threshold}
    """, conn)
    print(f"Edges > {threshold}: {count['count'][0]:,}")

conn.close()

Total edges: 2,133,010
Unique genes: 24,813
Edges > 0.25: 2,133,010
Edges > 0.5: 1,882,769
Edges > 0.6: 1,463,183
Edges > 0.7: 873,286
Edges > 0.8: 360,424
Edges > 0.9: 94,238
Edges > 0.95: 26,684
Edges > 0.97: 8,711
Edges > 0.99: 261


In [8]:
conn = sqlite3.connect('GTEx.db')

# Get just the unique weight values
unique_bad_weights = pd.read_sql("""
    SELECT DISTINCT weight
    FROM GTEx_network_age_adjusted 
    WHERE weight GLOB '*[a-zA-Z]*'
""", conn)

print("Unique weight values containing letters:")
for val in unique_bad_weights['weight']:
    print(f"  '{val}'")

conn.close()

Unique weight values containing letters:
  'weight'


In [9]:
conn = sqlite3.connect('GTEx.db')

# Get all table names
tables = pd.read_sql("""
    SELECT name FROM sqlite_master 
    WHERE type='table'
""", conn)

print("Tables in database:")
print(tables)

conn.close()

Tables in database:
                                name
0           GTEx_module_age_adjusted
1  GTEx_module_bigtable_age_adjusted
2  GTEx_module_REACTOME_age_adjusted
3          GTEx_network_age_adjusted


In [ ]:
conn = sqlite3.connect('GTEx.db')

# Get just the unique weight values
df = pd.read_sql("""
    SELECT * 
    FROM gene_embeddings
""", conn)

conn.close()
df['embedding'] = df['embedding'].apply(lambda x: np.array(json.loads(x)))



<class 'float'>


In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import UMAP

